
# 🧪 GPT-5-mini — Responses API 실습 (Colab)
이 노트북은 **GPT-5-mini**로 실습합니다.  
중점: **`instructions` + `input` 구조**, **`reasoning.effort`**, **`text.verbosity`**, **`max_output_tokens`**.

> ⚠️ GPT-5 계열은 `temperature`, `top_p`, `logprobs` **미지원**입니다. (요청 시 에러)



## 1) 환경 셋업
최신 OpenAI Python SDK 설치


In [ ]:

!pip -qU install openai>=1.55.0



## 2) API 키 설정
Colab 상단 **Settings → Variables**에 `OPENAI_API_KEY`를 저장해두었다고 가정합니다.  
없으면 아래 셀에서 입력을 요청합니다.


In [ ]:

import os
from getpass import getpass

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    pass

if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API key set.")



## 3) 클라이언트 초기화 & 모델 선택
- 기본 모델: **`gpt-5-mini`**
- (옵션) 접근 불가 시 **`gpt-4o-mini`**로 폴백


In [ ]:

from openai import OpenAI
client = OpenAI()

PRIMARY_MODEL = "gpt-5-mini"
FALLBACK_MODEL = "gpt-4o-mini"

def pick_model():
    try:
        # Minimal probe call
        _ = client.responses.create(
            model=PRIMARY_MODEL,
            reasoning={"effort": "minimal"},
            text={"verbosity": "low"},
            input="ping",
            max_output_tokens=5,
        )
        return PRIMARY_MODEL
    except Exception as e:
        print("[warn] gpt-5-mini not available → falling back to gpt-4o-mini")
        return FALLBACK_MODEL

MODEL = pick_model()
print("Using model:", MODEL)



## 4) Quickstart — `instructions` + `input`
Responses API의 권장 형식은 **`instructions`**(개략적 역할/스타일)과 **`input`**(사용자 질문) 분리입니다.


In [ ]:

response = client.responses.create(
    model=MODEL,
    reasoning={"effort": "low"},
    text={"verbosity": "medium"},
    instructions="You are a friendly instructor who explains AI concepts simply and precisely.",
    input="Explain what a Large Language Model is in 2-3 sentences.",
    max_output_tokens=150,
)
print(response.output_text)
try:
    print("\n[usage]", response.usage)
except Exception:
    pass



### 🏴‍☠️ 예시: 스타일 지시 (pirate tone)


In [ ]:

response = client.responses.create(
    model=MODEL,
    reasoning={"effort": "low"},
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript? Answer in one short sentence.",
    text={"verbosity": "low"},
    max_output_tokens=60,
)
print(response.output_text)



## 5) 헬퍼 함수 — `ask_mini()`
실험 반복을 쉽게 하기 위한 간단한 래퍼입니다.


In [ ]:

def ask_mini(
    user_input: str,
    instructions: str = "You are a helpful assistant.",
    reasoning_effort: str = "medium",
    verbosity: str = "medium",
    max_tokens: int = 300,
    model: str = MODEL,
):
    r = client.responses.create(
        model=model,
        reasoning={"effort": reasoning_effort},
        text={"verbosity": verbosity},
        instructions=instructions,
        input=user_input,
        max_output_tokens=max_tokens,
    )
    return r

demo = ask_mini("List three practical tips for writing better prompts.", verbosity="low", max_tokens=120)
print(demo.output_text)



## 6) 실습 1 — `reasoning.effort` 비교
같은 질문에 대해 `minimal / low / medium / high` 차이를 관찰하세요.


In [ ]:

question = "Explain overfitting to a beginner in <= 4 sentences."
for effort in ["minimal", "low", "medium", "high"]:
    print(f"\n=== reasoning.effort = {effort} ===")
    r = ask_mini(question, reasoning_effort=effort, verbosity="medium", max_tokens=220)
    print(r.output_text)



## 7) 실습 2 — `text.verbosity` 비교
`low / medium / high`에 따라 응답 길이와 설명 정도가 어떻게 다른지 비교하세요.


In [ ]:

question = "What is a binary search tree? Include a tiny Python snippet."
for v in ["low", "medium", "high"]:
    print(f"\n=== text.verbosity = {v} ===")
    r = ask_mini(question, reasoning_effort="low", verbosity=v, max_tokens=400)
    print(r.output_text[:1200])



## 8) 실습 3 — `max_output_tokens` 영향
출력 길이를 60 / 150 / 400으로 바꿔 보고, 어디서 잘리는지 확인하세요.


In [ ]:

question = "Give a step-by-step guide to implement a simple REST API server in Python using FastAPI."
for m in [60, 150, 400]:
    print(f"\n=== max_output_tokens = {m} ===")
    r = ask_mini(question, reasoning_effort="medium", verbosity="high", max_tokens=m)
    print(r.output_text[:1200])



## 9) 구조화(JSON) 응답
`response_format={"type":"json_object"}`로 파싱 가능한 출력 받기


In [ ]:

import json

prompt = "Summarize in <=2 sentences and extract 3 keywords: GPT-5-mini balances speed, cost, and capability."
resp = client.responses.create(
    model=MODEL,
    response_format={"type": "json_object"},
    instructions="Return a valid JSON object only. Keys: summary, keywords (list).",
    input=prompt,
    text={"verbosity": "low"},
    max_output_tokens=150,
)
print("Raw:", resp.output_text)
try:
    print("Parsed:", json.loads(resp.output_text))
except Exception as e:
    print("JSON parse error:", e)



## 10) (선택) 스트리밍 출력
토큰 단위로 점진적으로 표시합니다.


In [ ]:

from contextlib import contextmanager

@contextmanager
def response_stream(**kwargs):
    s = client.responses.stream(**kwargs)
    try:
        yield s
    finally:
        s.close()

with response_stream(
    model=MODEL,
    reasoning={"effort": "low"},
    text={"verbosity": "low"},
    instructions="Be concise.",
    input="Write a 5-line poem about on-device acceleration.",
    max_output_tokens=120,
) as stream:
    for event in stream:
        if event.type == "response.output_text.delta":
            print(event.delta, end="")
        elif event.type == "response.completed":
            print("\n\n[completed]")



---
## 🛠️ 트러블슈팅 / 유의사항
- **미지원 파라미터**: `temperature`, `top_p`, `logprobs` → **사용 금지**  
- **지연 단축**: `reasoning.effort` 낮추기, `text.verbosity="low"`, `max_output_tokens` 축소  
- **과금 관리**: 긴 응답/반복 호출을 피하고, 출력 길이를 제한하세요.
- **접근 권한 문제**: 본 노트북은 `gpt-5-mini` 실패 시 `gpt-4o-mini`로 폴백합니다.



---
_Last updated: 2025-11-06 14:19:35_  
© Ajou Univ. **모바일 컴퓨팅 특론** — GPT-5-mini Responses Lab
